### Implementation: Writer Identification using GMM-UBM supervectors

In [4]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
import scipy.spatial.distance as spdistance
import os
from sklearn import mixture
import math
import copy
import gzip
import pickle

In [2]:
### First step is to extract the features using ROOT_SIFT
class FeatureEx():
    
    def __init__(self):
        self.sift = cv2.xfeatures2d.SIFT_create()
        
    def extract(self, img):
        keypoints, img_features = self.sift.detectAndCompute(img, None)
        return keypoints, img_features
    
    #### For computing ROOT_SIFT feature
    def compute(self, img):
        img = cv2.imread(img, cv2.IMREAD_GRAYSCALE)
        if img is None:
            return "No image"
        
        kpts, descriptors = self.extract(img)
        
        ## Hellinger normalization
        descriptors += np.finfo(np.float32).eps
        descriptors /= np.sum(descriptors, axis=1)[:, np.newaxis]
        descriptors = np.sqrt(descriptors)
        
        return (kpts, descriptors)
        

In [ ]:
#### Testing the class FeatureEx
if __name__ == '__main__':
    fEx = FeatureEx()
    kpts1, des1 = fEx.compute('../icdar2013_benchmarking_dataset/001_1.tif')
    kpts2, des2 = fEx.compute('../icdar2013_benchmarking_dataset/001_2.tif')
    features = np.asarray(())
    features = des1
    print(features.shape)
    features = np.vstack((features, des2))
    print (features.shape)

In [3]:
def get_feature_matrix(path, max_desc=150000, max_desc_per_file=150):
    'path denotes the path to the image files'
    fEx = FeatureEx() #creating Feature extraction object
    
    directory = path
    
    # features is a matrix of features(row-wise)
    features = np.asarray(())
    
    for file in os.listdir(directory):
        print (file)
        if file.endswith(".tif"): 
            print(os.path.join(directory, file))
            filePath = os.path.join(directory, file)
            
            kpts, descriptors = fEx.compute(filePath)
            if descriptors.shape[0] > max_desc_per_file:
                'select the random max_desc_per_file from the descriptors'
                idx = np.random.randint(descriptors.shape[0], size=max_desc_per_file)
                descriptors = descriptors[idx, :]
                
            if features.size == 0:
                features = descriptors
            else:
                if features.shape[0] + descriptors.shape[0] <= max_desc:
                    features = np.vstack((features, descriptors))
                else:
                    break
        else:
            continue
    
    return features


In [4]:
def compute_gmm_params(features):
    'features is a matrix of features(row-wise) to train gmm model'
    
    ## using default number of clusters = 100 as is used in paper
    gmm = mixture.GaussianMixture(n_components=100, covariance_type='diag')
    gmm.fit(features)
    
    return gmm

In [6]:
# Let max_desc = 150000 for training ubm
# And max_desc_per_file = max_desc/no_of_files = 150000/1000 in this case
features = get_feature_matrix('../icdar2013_benchmarking_dataset/')
gmm = compute_gmm_params(features)

007_4.tif
../icdar2013_benchmarking_dataset/007_4.tif
123_3.tif
../icdar2013_benchmarking_dataset/123_3.tif
041_3.tif
../icdar2013_benchmarking_dataset/041_3.tif
005_2.tif
../icdar2013_benchmarking_dataset/005_2.tif
086_3.tif
../icdar2013_benchmarking_dataset/086_3.tif
213_3.tif
../icdar2013_benchmarking_dataset/213_3.tif
095_2.tif
../icdar2013_benchmarking_dataset/095_2.tif
159_4.tif
../icdar2013_benchmarking_dataset/159_4.tif
153_1.tif
../icdar2013_benchmarking_dataset/153_1.tif
081_4.tif
../icdar2013_benchmarking_dataset/081_4.tif
216_2.tif
../icdar2013_benchmarking_dataset/216_2.tif
081_3.tif
../icdar2013_benchmarking_dataset/081_3.tif
214_2.tif
../icdar2013_benchmarking_dataset/214_2.tif
156_2.tif
../icdar2013_benchmarking_dataset/156_2.tif
245_2.tif
../icdar2013_benchmarking_dataset/245_2.tif
246_1.tif
../icdar2013_benchmarking_dataset/246_1.tif
020_2.tif
../icdar2013_benchmarking_dataset/020_2.tif
194_4.tif
../icdar2013_benchmarking_dataset/194_4.tif
201_3.tif
../icdar2013_bench

011_3.tif
../icdar2013_benchmarking_dataset/011_3.tif
146_4.tif
../icdar2013_benchmarking_dataset/146_4.tif
056_4.tif
../icdar2013_benchmarking_dataset/056_4.tif
238_2.tif
../icdar2013_benchmarking_dataset/238_2.tif
047_4.tif
../icdar2013_benchmarking_dataset/047_4.tif
002_1.tif
../icdar2013_benchmarking_dataset/002_1.tif
016_2.tif
../icdar2013_benchmarking_dataset/016_2.tif
029_3.tif
../icdar2013_benchmarking_dataset/029_3.tif
039_4.tif
../icdar2013_benchmarking_dataset/039_4.tif
011_2.tif
../icdar2013_benchmarking_dataset/011_2.tif
222_1.tif
../icdar2013_benchmarking_dataset/222_1.tif
205_2.tif
../icdar2013_benchmarking_dataset/205_2.tif
167_3.tif
../icdar2013_benchmarking_dataset/167_3.tif
161_1.tif
../icdar2013_benchmarking_dataset/161_1.tif
225_3.tif
../icdar2013_benchmarking_dataset/225_3.tif
055_4.tif
../icdar2013_benchmarking_dataset/055_4.tif
087_3.tif
../icdar2013_benchmarking_dataset/087_3.tif
143_4.tif
../icdar2013_benchmarking_dataset/143_4.tif
248_4.tif
../icdar2013_bench

045_1.tif
../icdar2013_benchmarking_dataset/045_1.tif
023_2.tif
../icdar2013_benchmarking_dataset/023_2.tif
027_1.tif
../icdar2013_benchmarking_dataset/027_1.tif
146_3.tif
../icdar2013_benchmarking_dataset/146_3.tif
121_1.tif
../icdar2013_benchmarking_dataset/121_1.tif
001_4.tif
../icdar2013_benchmarking_dataset/001_4.tif
249_1.tif
../icdar2013_benchmarking_dataset/249_1.tif
075_3.tif
../icdar2013_benchmarking_dataset/075_3.tif
068_3.tif
../icdar2013_benchmarking_dataset/068_3.tif
196_2.tif
../icdar2013_benchmarking_dataset/196_2.tif
101_1.tif
../icdar2013_benchmarking_dataset/101_1.tif
176_3.tif
../icdar2013_benchmarking_dataset/176_3.tif
086_1.tif
../icdar2013_benchmarking_dataset/086_1.tif
180_3.tif
../icdar2013_benchmarking_dataset/180_3.tif
188_4.tif
../icdar2013_benchmarking_dataset/188_4.tif
061_4.tif
../icdar2013_benchmarking_dataset/061_4.tif
022_2.tif
../icdar2013_benchmarking_dataset/022_2.tif
001_3.tif
../icdar2013_benchmarking_dataset/001_3.tif
070_1.tif
../icdar2013_bench

105_2.tif
../icdar2013_benchmarking_dataset/105_2.tif
067_1.tif
../icdar2013_benchmarking_dataset/067_1.tif
052_4.tif
../icdar2013_benchmarking_dataset/052_4.tif
101_3.tif
../icdar2013_benchmarking_dataset/101_3.tif
113_3.tif
../icdar2013_benchmarking_dataset/113_3.tif
129_1.tif
../icdar2013_benchmarking_dataset/129_1.tif
155_4.tif
../icdar2013_benchmarking_dataset/155_4.tif
169_4.tif
../icdar2013_benchmarking_dataset/169_4.tif
150_3.tif
../icdar2013_benchmarking_dataset/150_3.tif
069_1.tif
../icdar2013_benchmarking_dataset/069_1.tif
175_2.tif
../icdar2013_benchmarking_dataset/175_2.tif
211_4.tif
../icdar2013_benchmarking_dataset/211_4.tif
241_4.tif
../icdar2013_benchmarking_dataset/241_4.tif
154_3.tif
../icdar2013_benchmarking_dataset/154_3.tif
053_2.tif
../icdar2013_benchmarking_dataset/053_2.tif
100_2.tif
../icdar2013_benchmarking_dataset/100_2.tif
009_3.tif
../icdar2013_benchmarking_dataset/009_3.tif
141_2.tif
../icdar2013_benchmarking_dataset/141_2.tif
088_3.tif
../icdar2013_bench

106_4.tif
../icdar2013_benchmarking_dataset/106_4.tif
083_4.tif
../icdar2013_benchmarking_dataset/083_4.tif
035_4.tif
../icdar2013_benchmarking_dataset/035_4.tif
058_4.tif
../icdar2013_benchmarking_dataset/058_4.tif
010_2.tif
../icdar2013_benchmarking_dataset/010_2.tif
075_2.tif
../icdar2013_benchmarking_dataset/075_2.tif
141_3.tif
../icdar2013_benchmarking_dataset/141_3.tif
127_3.tif
../icdar2013_benchmarking_dataset/127_3.tif
065_1.tif
../icdar2013_benchmarking_dataset/065_1.tif
051_3.tif
../icdar2013_benchmarking_dataset/051_3.tif
015_1.tif
../icdar2013_benchmarking_dataset/015_1.tif
116_4.tif
../icdar2013_benchmarking_dataset/116_4.tif
236_1.tif
../icdar2013_benchmarking_dataset/236_1.tif
040_2.tif
../icdar2013_benchmarking_dataset/040_2.tif
165_4.tif
../icdar2013_benchmarking_dataset/165_4.tif
003_4.tif
../icdar2013_benchmarking_dataset/003_4.tif
104_3.tif
../icdar2013_benchmarking_dataset/104_3.tif
082_1.tif
../icdar2013_benchmarking_dataset/082_1.tif
244_3.tif
../icdar2013_bench

224_1.tif
../icdar2013_benchmarking_dataset/224_1.tif
163_4.tif
../icdar2013_benchmarking_dataset/163_4.tif
085_2.tif
../icdar2013_benchmarking_dataset/085_2.tif
215_2.tif
../icdar2013_benchmarking_dataset/215_2.tif
250_1.tif
../icdar2013_benchmarking_dataset/250_1.tif
003_2.tif
../icdar2013_benchmarking_dataset/003_2.tif
229_1.tif
../icdar2013_benchmarking_dataset/229_1.tif
064_2.tif
../icdar2013_benchmarking_dataset/064_2.tif
048_1.tif
../icdar2013_benchmarking_dataset/048_1.tif
086_2.tif
../icdar2013_benchmarking_dataset/086_2.tif
189_2.tif
../icdar2013_benchmarking_dataset/189_2.tif
120_4.tif
../icdar2013_benchmarking_dataset/120_4.tif
222_3.tif
../icdar2013_benchmarking_dataset/222_3.tif
052_1.tif
../icdar2013_benchmarking_dataset/052_1.tif
159_3.tif
../icdar2013_benchmarking_dataset/159_3.tif
021_2.tif
../icdar2013_benchmarking_dataset/021_2.tif
237_1.tif
../icdar2013_benchmarking_dataset/237_1.tif
010_1.tif
../icdar2013_benchmarking_dataset/010_1.tif
059_2.tif
../icdar2013_bench

082_3.tif
../icdar2013_benchmarking_dataset/082_3.tif
026_4.tif
../icdar2013_benchmarking_dataset/026_4.tif
018_3.tif
../icdar2013_benchmarking_dataset/018_3.tif
041_1.tif
../icdar2013_benchmarking_dataset/041_1.tif
016_4.tif
../icdar2013_benchmarking_dataset/016_4.tif
168_2.tif
../icdar2013_benchmarking_dataset/168_2.tif
041_4.tif
../icdar2013_benchmarking_dataset/041_4.tif
130_1.tif
../icdar2013_benchmarking_dataset/130_1.tif
141_1.tif
../icdar2013_benchmarking_dataset/141_1.tif
092_3.tif
../icdar2013_benchmarking_dataset/092_3.tif
050_1.tif
../icdar2013_benchmarking_dataset/050_1.tif
013_1.tif
../icdar2013_benchmarking_dataset/013_1.tif
044_3.tif
../icdar2013_benchmarking_dataset/044_3.tif
034_2.tif
../icdar2013_benchmarking_dataset/034_2.tif
173_2.tif
../icdar2013_benchmarking_dataset/173_2.tif
104_4.tif
../icdar2013_benchmarking_dataset/104_4.tif
067_3.tif
../icdar2013_benchmarking_dataset/067_3.tif
118_4.tif
../icdar2013_benchmarking_dataset/118_4.tif
205_4.tif
../icdar2013_bench

In [7]:
def adaptMAP(data, gmm, posteriors, relevance = 16, update='wmc'):

    """
    Adapt new data to a given gmm with a specific
    relevance factor.
    Reference: D.A.Reynolds et al. 'Speaker Verification Using Adapted Gaussian
    Mixture Models'
    """

    sum_post = np.sum(posteriors, axis=0) # (N_component x ,)
    
    nd = len(gmm.weights_) # number of components / gaussians
    fd = data.shape[1] # feature dimension

    data_square = data * data

    def loop(i):
        means_ = posteriors[:,i].reshape(1,-1).dot(data)
        covs_ = posteriors[:,i].reshape(1,-1).dot(data_square)
        return means_, covs_

    means, covs = zip( *map(loop, range(nd)))

    means = np.array(means).reshape(nd, fd)
    covs = np.array(covs).reshape(nd,fd)

    # add some small number
    means += np.finfo(float).eps
    covs += np.finfo(float).eps

    # normalize them
    means /= sum_post.reshape(-1,1) + np.finfo(float).eps
    covs /= sum_post.reshape(-1,1) + np.finfo(float).eps

    # now combine the two estimates using the relevance factor
    # i.e. interpolation controlled by relevance factor
    def combine(i):
        alpha = sum_post[i] / (sum_post[i] + relevance)

        # update priors
        if 'w' in update:
            weights_ = ( (alpha * sum_post[i]) / float(len(data)) ) \
                    + ( (1.0 - alpha) * gmm.weights_[i] )
        else:
            weights_ = copy.deepcopy(gmm.weights_[i])
    
        # update means
        if 'm' in update:
            means_ = alpha * means[i] \
                    + ( (1.0 -alpha) * gmm.means_[i] )
        else:
            means_ = copy.deepcopy(gmm.means_[i])
        # update covariance matrix
        if 'c' in update:
            covs_ = alpha * covs[i] \
                    + (1.0 - alpha) * (gmm.covariances_[i] + \
                                       gmm.means_[i] * gmm.means_[i])\
                    - (means_ * means_) # careful, this is means_ not means_[i], 
                                        # since we are in that specific
                                        # component computation already!
        else:
            covs_ = copy.deepcopy(gmm.covariances_[i])

        return weights_, means_, covs_

    weights, means, covs = zip( *map(combine, range(nd)) )

    weights = np.array(weights)
    means = np.array(means)
    covs = np.array(covs)

    # let weights sum to 1
    if 'w' in update:
        weights /= weights.sum() + np.finfo(float).eps

    # create new mixture  
    adapted_gmm = mixture.GaussianMixture(nd)
    # and assign mean, cov, priors to it
    adapted_gmm.weights_ = weights
    adapted_gmm.means_ = means
    adapted_gmm.covariances_ = covs
    # adapted_gmm._set_covariances( covs ) # this variant checks the covariances 

    
    return adapted_gmm

In [8]:
def supervectorStacking(gmm, update='wmc'):
    """
    form supervector, optionally normalize each component
    """
    
    enc = []
    
    if 'm' in update:
        enc_m = gmm.means_
        enc.append(enc_m.reshape(1,-1))

    if 'c' in update:
        enc_c = gmm.covariances_
        enc.append(enc_c.reshape(1,-1))

    if 'w' in update:
        enc_w = gmm.weights_
        enc.append(enc_w.reshape(1,-1))

    enc = np.concatenate(enc, axis=1)
    
    # Normalize enc
    enc = np.sqrt(enc)
    enc = preprocessing.normalize(enc)

    return enc

In [9]:
def supervector(gmm, features, posteriors, relevance=16, update='wmc'):
    'Returns stacked supervector for the particular document'
    'features are the feature matrix of that particular document'
    scribe_gmm = adaptMAP(features, gmm, posteriors, relevance, update)
    
    return supervectorStacking(scribe_gmm, update)

In [10]:
'''
    For adapting to a document W, calculate it's features
    and then using that features compute posteriors
    and then call supervector function to get the supervector corresponding to document W
'''

fEx = FeatureEx()
filePath = '../icdar2013_benchmarking_dataset/001_1.tif'
kpts, data = fEx.compute(filePath)

posteriors = gmm.predict_proba(data)
enc = supervector(gmm, data, posteriors)
print(enc)

[[0.00572797 0.00871033 0.01082185 ... 0.0037791  0.00436174 0.00347353]]


In [24]:
def ubm_adaptation(path, outDir, gmm):
    'path denotes the path to the image files'
    fEx = FeatureEx() # creating Feature extraction object
    
    directory = path
    
    # features is a matrix of features(row-wise)

    for file in os.listdir(directory):
        print (file)
        if file.endswith(".tif"): 
            print(os.path.join(directory, file))
            fp = os.path.join(directory, file)
            print(fp)
            kpts, data = fEx.compute(fp)
            
            posteriors = gmm.predict_proba(data)
            enc = supervector(gmm, data, posteriors)
            print(enc)
            
            #### save the encoding in a folder outDir as pickle file
            fileName, file_ext = os.path.splitext(file)
            
            if not os.path.exists(outDir):
                os.makedirs(outDir)
            
            filepath = os.path.join(outDir, fileName + '.pkl.gz')
            print(filepath)
            with gzip.open(filepath, 'wb') as f:
                print(f)
                pickle.dump(enc, f, -1)
                
        else:
            continue


In [25]:
ubm_adaptation('../experimental_dataset_2013/', '../outTestVoc', gmm)

123_3.tif
../experimental_dataset_2013/123_3.tif
../experimental_dataset_2013/123_3.tif
[[0.00439875 0.00762963 0.01072019 ... 0.00377986 0.00473885 0.0032714 ]]
../outTestVoc/123_3.pkl.gz
<gzip _io.BufferedWriter name='../outTestVoc/123_3.pkl.gz' 0x7f5d800dbcf8>
041_3.tif
../experimental_dataset_2013/041_3.tif
../experimental_dataset_2013/041_3.tif
[[0.00473437 0.00781227 0.01027514 ... 0.004069   0.00418095 0.00328866]]
../outTestVoc/041_3.pkl.gz
<gzip _io.BufferedWriter name='../outTestVoc/041_3.pkl.gz' 0x7f5d800db518>
086_3.tif
../experimental_dataset_2013/086_3.tif
../experimental_dataset_2013/086_3.tif
[[0.00440543 0.00780229 0.0103727  ... 0.00354991 0.00333568 0.002772  ]]
../outTestVoc/086_3.pkl.gz
<gzip _io.BufferedWriter name='../outTestVoc/086_3.pkl.gz' 0x7f5d800dbb70>
095_2.tif
../experimental_dataset_2013/095_2.tif
../experimental_dataset_2013/095_2.tif
[[0.00445864 0.00687138 0.00909987 ... 0.00353626 0.00474938 0.00280139]]
../outTestVoc/095_2.pkl.gz
<gzip _io.BufferedW

[[0.00533114 0.0079821  0.0100067  ... 0.00382227 0.00405498 0.00273682]]
../outTestVoc/114_2.pkl.gz
<gzip _io.BufferedWriter name='../outTestVoc/114_2.pkl.gz' 0x7f5d800db518>
064_1.tif
../experimental_dataset_2013/064_1.tif
../experimental_dataset_2013/064_1.tif
[[0.0044901  0.0080361  0.01100967 ... 0.00948672 0.00324856 0.00285987]]
../outTestVoc/064_1.pkl.gz
<gzip _io.BufferedWriter name='../outTestVoc/064_1.pkl.gz' 0x7f5d800dbb70>
090_4.tif
../experimental_dataset_2013/090_4.tif
../experimental_dataset_2013/090_4.tif
[[0.00381641 0.00620786 0.00897193 ... 0.00352968 0.00421419 0.00287691]]
../outTestVoc/090_4.pkl.gz
<gzip _io.BufferedWriter name='../outTestVoc/090_4.pkl.gz' 0x7f5d800dbc18>
077_2.tif
../experimental_dataset_2013/077_2.tif
../experimental_dataset_2013/077_2.tif
[[0.00499541 0.00834134 0.01026379 ... 0.00353396 0.00346154 0.00293465]]
../outTestVoc/077_2.pkl.gz
<gzip _io.BufferedWriter name='../outTestVoc/077_2.pkl.gz' 0x7f5d800db978>
027_4.tif
../experimental_datase

[[0.0057414  0.00905846 0.01069884 ... 0.00508257 0.00388162 0.00346785]]
../outTestVoc/126_4.pkl.gz
<gzip _io.BufferedWriter name='../outTestVoc/126_4.pkl.gz' 0x7f5d800dbc18>
038_1.tif
../experimental_dataset_2013/038_1.tif
../experimental_dataset_2013/038_1.tif
[[0.00618148 0.00911387 0.00970808 ... 0.00379159 0.00367891 0.00288209]]
../outTestVoc/038_1.pkl.gz
<gzip _io.BufferedWriter name='../outTestVoc/038_1.pkl.gz' 0x7f5d800dba90>
029_2.tif
../experimental_dataset_2013/029_2.tif
../experimental_dataset_2013/029_2.tif
[[0.00537045 0.00812847 0.00914889 ... 0.00369084 0.00341823 0.00321209]]
../outTestVoc/029_2.pkl.gz
<gzip _io.BufferedWriter name='../outTestVoc/029_2.pkl.gz' 0x7f5d800dbb70>
126_1.tif
../experimental_dataset_2013/126_1.tif
../experimental_dataset_2013/126_1.tif
[[0.00508019 0.00813343 0.01022729 ... 0.00359494 0.00373155 0.00351447]]
../outTestVoc/126_1.pkl.gz
<gzip _io.BufferedWriter name='../outTestVoc/126_1.pkl.gz' 0x7f5d800dbba8>
090_3.tif
../experimental_datase

[[0.00587407 0.00727291 0.01043027 ... 0.00769411 0.00338381 0.00263679]]
../outTestVoc/053_4.pkl.gz
<gzip _io.BufferedWriter name='../outTestVoc/053_4.pkl.gz' 0x7f5d800dbb70>
033_1.tif
../experimental_dataset_2013/033_1.tif
../experimental_dataset_2013/033_1.tif
[[0.00580499 0.00836882 0.01028682 ... 0.0038209  0.00433808 0.00363379]]
../outTestVoc/033_1.pkl.gz
<gzip _io.BufferedWriter name='../outTestVoc/033_1.pkl.gz' 0x7f5d800db128>
101_2.tif
../experimental_dataset_2013/101_2.tif
../experimental_dataset_2013/101_2.tif
[[0.00452261 0.00779382 0.01125148 ... 0.00414506 0.00384927 0.00309858]]
../outTestVoc/101_2.pkl.gz
<gzip _io.BufferedWriter name='../outTestVoc/101_2.pkl.gz' 0x7f5d800dbba8>
080_2.tif
../experimental_dataset_2013/080_2.tif
../experimental_dataset_2013/080_2.tif
[[0.00529759 0.00840903 0.00955264 ... 0.00624684 0.00407039 0.00267567]]
../outTestVoc/080_2.pkl.gz
<gzip _io.BufferedWriter name='../outTestVoc/080_2.pkl.gz' 0x7f5d800db978>
106_3.tif
../experimental_datase

[[0.0050563  0.00806794 0.00968969 ... 0.00394105 0.00354667 0.00322701]]
../outTestVoc/066_2.pkl.gz
<gzip _io.BufferedWriter name='../outTestVoc/066_2.pkl.gz' 0x7f5d800dbb70>
075_1.tif
../experimental_dataset_2013/075_1.tif
../experimental_dataset_2013/075_1.tif
[[0.00538779 0.00902395 0.01038362 ... 0.00416864 0.00323834 0.00267408]]
../outTestVoc/075_1.pkl.gz
<gzip _io.BufferedWriter name='../outTestVoc/075_1.pkl.gz' 0x7f5d800dbba8>
078_3.tif
../experimental_dataset_2013/078_3.tif
../experimental_dataset_2013/078_3.tif
[[0.00544768 0.00831594 0.0107933  ... 0.00382013 0.00371551 0.00272139]]
../outTestVoc/078_3.pkl.gz
<gzip _io.BufferedWriter name='../outTestVoc/078_3.pkl.gz' 0x7f5d800dbc18>
089_2.tif
../experimental_dataset_2013/089_2.tif
../experimental_dataset_2013/089_2.tif
[[0.00448066 0.00771924 0.00983557 ... 0.00604575 0.00349242 0.0026955 ]]
../outTestVoc/089_2.pkl.gz
<gzip _io.BufferedWriter name='../outTestVoc/089_2.pkl.gz' 0x7f5d800dba90>
034_1.tif
../experimental_datase

...

[[0.00610548 0.0090835  0.01081073 ... 0.00372221 0.00412517 0.00389115]]
../outTestVoc/051_3.pkl.gz
<gzip _io.BufferedWriter name='../outTestVoc/051_3.pkl.gz' 0x7f5d800dba90>
116_4.tif
../experimental_dataset_2013/116_4.tif
../experimental_dataset_2013/116_4.tif
[[0.00545032 0.00778463 0.00943469 ... 0.00781537 0.00322443 0.0026212 ]]
../outTestVoc/116_4.pkl.gz
<gzip _io.BufferedWriter name='../outTestVoc/116_4.pkl.gz' 0x7f5d861ada90>
040_2.tif
../experimental_dataset_2013/040_2.tif
../experimental_dataset_2013/040_2.tif
[[0.00574827 0.00894018 0.01017074 ... 0.00387559 0.00393998 0.00350818]]
../outTestVoc/040_2.pkl.gz
<gzip _io.BufferedWriter name='../outTestVoc/040_2.pkl.gz' 0x7f5d800dbcf8>
104_3.tif
../experimental_dataset_2013/104_3.tif
../experimental_dataset_2013/104_3.tif
[[0.00498086 0.00774136 0.01064523 ... 0.00357115 0.00417562 0.0027576 ]]
../outTestVoc/104_3.pkl.gz
<gzip _io.BufferedWriter name='../outTestVoc/104_3.pkl.gz' 0x7f5d800dbba8>
082_1.tif
../experimental_datase

[[0.00471775 0.00759122 0.00990207 ... 0.00372937 0.00506829 0.00273418]]
../outTestVoc/028_4.pkl.gz
<gzip _io.BufferedWriter name='../outTestVoc/028_4.pkl.gz' 0x7f5d800db978>
123_1.tif
../experimental_dataset_2013/123_1.tif
../experimental_dataset_2013/123_1.tif
[[0.00488643 0.00805914 0.01039403 ... 0.0039186  0.00497166 0.00383473]]
../outTestVoc/123_1.pkl.gz
<gzip _io.BufferedWriter name='../outTestVoc/123_1.pkl.gz' 0x7f5d800dbcf8>
067_2.tif
../experimental_dataset_2013/067_2.tif
../experimental_dataset_2013/067_2.tif
[[0.00615031 0.00868685 0.00970087 ... 0.00370158 0.00512898 0.00272984]]
../outTestVoc/067_2.pkl.gz
<gzip _io.BufferedWriter name='../outTestVoc/067_2.pkl.gz' 0x7f5d861ada90>
065_2.tif
../experimental_dataset_2013/065_2.tif
../experimental_dataset_2013/065_2.tif
[[0.00530912 0.00773563 0.0090535  ... 0.00348066 0.00355421 0.00281914]]
../outTestVoc/065_2.pkl.gz
<gzip _io.BufferedWriter name='../outTestVoc/065_2.pkl.gz' 0x7f5d800dbb70>
117_3.tif
../experimental_datase

[[0.00526627 0.00844373 0.00991892 ... 0.00369179 0.00395355 0.00273798]]
../outTestVoc/059_2.pkl.gz
<gzip _io.BufferedWriter name='../outTestVoc/059_2.pkl.gz' 0x7f5d861ada90>
037_4.tif
../experimental_dataset_2013/037_4.tif
../experimental_dataset_2013/037_4.tif
[[0.00506695 0.00844266 0.00978954 ... 0.00354274 0.00440672 0.0028884 ]]
../outTestVoc/037_4.pkl.gz
<gzip _io.BufferedWriter name='../outTestVoc/037_4.pkl.gz' 0x7f5d800dba90>
059_4.tif
../experimental_dataset_2013/059_4.tif
../experimental_dataset_2013/059_4.tif
[[0.00626341 0.00859297 0.00988916 ... 0.0054107  0.00340198 0.00293223]]
../outTestVoc/059_4.pkl.gz
<gzip _io.BufferedWriter name='../outTestVoc/059_4.pkl.gz' 0x7f5d800db978>
085_1.tif
../experimental_dataset_2013/085_1.tif
../experimental_dataset_2013/085_1.tif
[[0.00585071 0.00813225 0.00992598 ... 0.00369593 0.00418342 0.00299574]]
../outTestVoc/085_1.pkl.gz
<gzip _io.BufferedWriter name='../outTestVoc/085_1.pkl.gz' 0x7f5d861ada90>
123_2.tif
../experimental_datase

[[0.00563753 0.00849701 0.01056755 ... 0.00402927 0.00570072 0.00297233]]
../outTestVoc/076_3.pkl.gz
<gzip _io.BufferedWriter name='../outTestVoc/076_3.pkl.gz' 0x7f5d800db978>
076_4.tif
../experimental_dataset_2013/076_4.tif
../experimental_dataset_2013/076_4.tif
[[0.00551278 0.00862954 0.01009784 ... 0.0040804  0.00465181 0.00310114]]
../outTestVoc/076_4.pkl.gz
<gzip _io.BufferedWriter name='../outTestVoc/076_4.pkl.gz' 0x7f5d861ada90>
120_2.tif
../experimental_dataset_2013/120_2.tif
../experimental_dataset_2013/120_2.tif
[[0.00622958 0.00959133 0.01024832 ... 0.00429828 0.00373733 0.00332904]]
../outTestVoc/120_2.pkl.gz
<gzip _io.BufferedWriter name='../outTestVoc/120_2.pkl.gz' 0x7f5d800dba90>
112_3.tif
../experimental_dataset_2013/112_3.tif
../experimental_dataset_2013/112_3.tif
[[0.00499886 0.00774679 0.01009175 ... 0.00360786 0.00457909 0.00312039]]
../outTestVoc/112_3.pkl.gz
<gzip _io.BufferedWriter name='../outTestVoc/112_3.pkl.gz' 0x7f5d800dbc18>
028_2.tif
../experimental_datase

[[0.00597674 0.00869802 0.01041912 ... 0.00359697 0.00353184 0.00416162]]
../outTestVoc/033_4.pkl.gz
<gzip _io.BufferedWriter name='../outTestVoc/033_4.pkl.gz' 0x7f5d800db978>
080_3.tif
../experimental_dataset_2013/080_3.tif
../experimental_dataset_2013/080_3.tif
[[0.00454041 0.00811927 0.01048571 ... 0.00657837 0.00330611 0.00278702]]
../outTestVoc/080_3.pkl.gz
<gzip _io.BufferedWriter name='../outTestVoc/080_3.pkl.gz' 0x7f5d800dbc18>
115_3.tif
../experimental_dataset_2013/115_3.tif
../experimental_dataset_2013/115_3.tif
[[0.00611111 0.00915655 0.00981254 ... 0.0041859  0.0037882  0.00280524]]
../outTestVoc/115_3.pkl.gz
<gzip _io.BufferedWriter name='../outTestVoc/115_3.pkl.gz' 0x7f5d861ada90>
114_3.tif
../experimental_dataset_2013/114_3.tif
../experimental_dataset_2013/114_3.tif
[[0.00466109 0.00745827 0.01008053 ... 0.00361378 0.00451393 0.00279776]]
../outTestVoc/114_3.pkl.gz
<gzip _io.BufferedWriter name='../outTestVoc/114_3.pkl.gz' 0x7f5d800db518>
098_1.tif
../experimental_datase

In [16]:
file = '001_1.tif'
fileName, file_ext = os.path.splitext(file)
print(fileName)
# enc = np.array([1, 2, 3])
# enc = enc.reshape(1, 3)
# directory = '../outVoc'
# if not os.path.exists(directory):
#     os.makedirs(directory)
# filePath = '../outVoc/file.pkl.gz'
# with gzip.open(filePath, 'wb') as f:
#     pickle.dump(enc, f, -1)

001_1


In [27]:
ubm_adaptation('../icdar2013_benchmarking_dataset/', '../outTrainVoc', gmm)

007_4.tif
../icdar2013_benchmarking_dataset/007_4.tif
../icdar2013_benchmarking_dataset/007_4.tif
[[0.00587518 0.00866155 0.01122732 ... 0.0035322  0.00428738 0.00297838]]
../outTrainVoc/007_4.pkl.gz
<gzip _io.BufferedWriter name='../outTrainVoc/007_4.pkl.gz' 0x7f5d800db4e0>
123_3.tif
../icdar2013_benchmarking_dataset/123_3.tif
../icdar2013_benchmarking_dataset/123_3.tif
[[0.00556191 0.0083634  0.01022144 ... 0.00365021 0.00324158 0.00269829]]
../outTrainVoc/123_3.pkl.gz
<gzip _io.BufferedWriter name='../outTrainVoc/123_3.pkl.gz' 0x7f5d800dbac8>
041_3.tif
../icdar2013_benchmarking_dataset/041_3.tif
../icdar2013_benchmarking_dataset/041_3.tif
[[0.00450674 0.00771394 0.01091981 ... 0.00363101 0.00362315 0.00273473]]
../outTrainVoc/041_3.pkl.gz
<gzip _io.BufferedWriter name='../outTrainVoc/041_3.pkl.gz' 0x7f5d800dbf60>
005_2.tif
../icdar2013_benchmarking_dataset/005_2.tif
../icdar2013_benchmarking_dataset/005_2.tif
[[0.00579422 0.00913577 0.01067436 ... 0.00347441 0.00361394 0.00413663]]


[[0.00583508 0.00826987 0.010938   ... 0.00352679 0.0036582  0.00439941]]
../outTrainVoc/111_3.pkl.gz
<gzip _io.BufferedWriter name='../outTrainVoc/111_3.pkl.gz' 0x7f5d800db940>
206_3.tif
../icdar2013_benchmarking_dataset/206_3.tif
../icdar2013_benchmarking_dataset/206_3.tif
[[0.00597421 0.00871666 0.01034221 ... 0.00652552 0.00377901 0.00334469]]
../outTrainVoc/206_3.pkl.gz
<gzip _io.BufferedWriter name='../outTrainVoc/206_3.pkl.gz' 0x7f5d800db4e0>
189_1.tif
../icdar2013_benchmarking_dataset/189_1.tif
../icdar2013_benchmarking_dataset/189_1.tif
[[0.00584172 0.00742147 0.0102765  ... 0.00396465 0.00381444 0.00371847]]
../outTrainVoc/189_1.pkl.gz
<gzip _io.BufferedWriter name='../outTrainVoc/189_1.pkl.gz' 0x7f5d800db128>
215_1.tif
../icdar2013_benchmarking_dataset/215_1.tif
../icdar2013_benchmarking_dataset/215_1.tif
[[0.0057114  0.0080785  0.00976417 ... 0.00558575 0.00417473 0.00269242]]
../outTrainVoc/215_1.pkl.gz
<gzip _io.BufferedWriter name='../outTrainVoc/215_1.pkl.gz' 0x7f5d800d

[[0.00542366 0.00900037 0.01036072 ... 0.00490869 0.00347628 0.00300331]]
../outTrainVoc/176_4.pkl.gz
<gzip _io.BufferedWriter name='../outTrainVoc/176_4.pkl.gz' 0x7f5d800db3c8>
237_3.tif
../icdar2013_benchmarking_dataset/237_3.tif
../icdar2013_benchmarking_dataset/237_3.tif
[[0.0049548  0.00700488 0.00963312 ... 0.00341369 0.003866   0.00378411]]
../outTrainVoc/237_3.pkl.gz
<gzip _io.BufferedWriter name='../outTrainVoc/237_3.pkl.gz' 0x7f5d800db4e0>
190_4.tif
../icdar2013_benchmarking_dataset/190_4.tif
../icdar2013_benchmarking_dataset/190_4.tif
[[0.0059224  0.0079912  0.00990942 ... 0.00479388 0.00324267 0.00263671]]
../outTrainVoc/190_4.pkl.gz
<gzip _io.BufferedWriter name='../outTrainVoc/190_4.pkl.gz' 0x7f5d800dbf60>
046_2.tif
../icdar2013_benchmarking_dataset/046_2.tif
../icdar2013_benchmarking_dataset/046_2.tif
[[0.00466069 0.00859542 0.01080936 ... 0.00401424 0.00359305 0.00370839]]
../outTrainVoc/046_2.pkl.gz
<gzip _io.BufferedWriter name='../outTrainVoc/046_2.pkl.gz' 0x7f5d800d

[[0.00591843 0.00870465 0.00969774 ... 0.00397617 0.00484748 0.00287306]]
../outTrainVoc/119_2.pkl.gz
<gzip _io.BufferedWriter name='../outTrainVoc/119_2.pkl.gz' 0x7f5d800dbf60>
114_2.tif
../icdar2013_benchmarking_dataset/114_2.tif
../icdar2013_benchmarking_dataset/114_2.tif
[[0.00625131 0.00853101 0.00892271 ... 0.00344145 0.00373698 0.00291957]]
../outTrainVoc/114_2.pkl.gz
<gzip _io.BufferedWriter name='../outTrainVoc/114_2.pkl.gz' 0x7f5d800db3c8>
216_1.tif
../icdar2013_benchmarking_dataset/216_1.tif
../icdar2013_benchmarking_dataset/216_1.tif
[[0.0056125  0.00785478 0.01001115 ... 0.00356544 0.0050544  0.00396695]]
../outTrainVoc/216_1.pkl.gz
<gzip _io.BufferedWriter name='../outTrainVoc/216_1.pkl.gz' 0x7f5d800db9b0>
064_1.tif
../icdar2013_benchmarking_dataset/064_1.tif
../icdar2013_benchmarking_dataset/064_1.tif
[[0.00622337 0.00768856 0.00969947 ... 0.00762312 0.00317584 0.00298177]]
../outTrainVoc/064_1.pkl.gz
<gzip _io.BufferedWriter name='../outTrainVoc/064_1.pkl.gz' 0x7f5d800d

[[0.00622439 0.00828862 0.00910297 ... 0.00362246 0.00367662 0.00418592]]
../outTrainVoc/169_2.pkl.gz
<gzip _io.BufferedWriter name='../outTrainVoc/169_2.pkl.gz' 0x7f5d800dbf60>
135_1.tif
../icdar2013_benchmarking_dataset/135_1.tif
../icdar2013_benchmarking_dataset/135_1.tif
[[0.00494512 0.00771999 0.01013837 ... 0.00390906 0.00367729 0.00266294]]
../outTrainVoc/135_1.pkl.gz
<gzip _io.BufferedWriter name='../outTrainVoc/135_1.pkl.gz' 0x7f5d800db518>
149_2.tif
../icdar2013_benchmarking_dataset/149_2.tif
../icdar2013_benchmarking_dataset/149_2.tif
[[0.00654629 0.00931888 0.01037842 ... 0.00431203 0.00349024 0.00355533]]
../outTrainVoc/149_2.pkl.gz
<gzip _io.BufferedWriter name='../outTrainVoc/149_2.pkl.gz' 0x7f5d800db3c8>
007_2.tif
../icdar2013_benchmarking_dataset/007_2.tif
../icdar2013_benchmarking_dataset/007_2.tif
[[0.00592246 0.00860226 0.00972671 ... 0.00360664 0.00477467 0.00273963]]
../outTrainVoc/007_2.pkl.gz
<gzip _io.BufferedWriter name='../outTrainVoc/007_2.pkl.gz' 0x7f5d800d

...

[[0.00476234 0.00772754 0.01096446 ... 0.00812173 0.00315269 0.0025855 ]]
../outTrainVoc/076_3.pkl.gz
<gzip _io.BufferedWriter name='../outTrainVoc/076_3.pkl.gz' 0x7f5d800db128>
222_2.tif
../icdar2013_benchmarking_dataset/222_2.tif
../icdar2013_benchmarking_dataset/222_2.tif
[[0.00485004 0.00733886 0.01019826 ... 0.00388506 0.00466149 0.00275792]]
../outTrainVoc/222_2.pkl.gz
<gzip _io.BufferedWriter name='../outTrainVoc/222_2.pkl.gz' 0x7f5d800dbac8>
134_2.tif
../icdar2013_benchmarking_dataset/134_2.tif
../icdar2013_benchmarking_dataset/134_2.tif
[[0.00510342 0.00848614 0.01054197 ... 0.00387377 0.00411237 0.00366585]]
../outTrainVoc/134_2.pkl.gz
<gzip _io.BufferedWriter name='../outTrainVoc/134_2.pkl.gz' 0x7f5d800db4e0>
199_1.tif
../icdar2013_benchmarking_dataset/199_1.tif
../icdar2013_benchmarking_dataset/199_1.tif
[[0.00565028 0.00793719 0.00989142 ... 0.0058026  0.00333754 0.00319046]]
../outTrainVoc/199_1.pkl.gz
<gzip _io.BufferedWriter name='../outTrainVoc/199_1.pkl.gz' 0x7f5d800d

[[0.00420609 0.00784392 0.01002031 ... 0.00360958 0.00426166 0.00273255]]
../outTrainVoc/091_1.pkl.gz
<gzip _io.BufferedWriter name='../outTrainVoc/091_1.pkl.gz' 0x7f5d800db9b0>
163_1.tif
../icdar2013_benchmarking_dataset/163_1.tif
../icdar2013_benchmarking_dataset/163_1.tif
[[0.00576598 0.00891385 0.01018113 ... 0.00353243 0.00369527 0.00270457]]
../outTrainVoc/163_1.pkl.gz
<gzip _io.BufferedWriter name='../outTrainVoc/163_1.pkl.gz' 0x7f5d800db3c8>
211_1.tif
../icdar2013_benchmarking_dataset/211_1.tif
../icdar2013_benchmarking_dataset/211_1.tif
[[0.00441279 0.00858228 0.01087746 ... 0.0036009  0.0032711  0.00257606]]
../outTrainVoc/211_1.pkl.gz
<gzip _io.BufferedWriter name='../outTrainVoc/211_1.pkl.gz' 0x7f5d800db518>
178_1.tif
../icdar2013_benchmarking_dataset/178_1.tif
../icdar2013_benchmarking_dataset/178_1.tif
[[0.00617089 0.00894377 0.01053209 ... 0.0064027  0.00339942 0.00296864]]
../outTrainVoc/178_1.pkl.gz
<gzip _io.BufferedWriter name='../outTrainVoc/178_1.pkl.gz' 0x7f5d800d

[[0.00580371 0.00686456 0.01005027 ... 0.00553363 0.00404116 0.00276757]]
../outTrainVoc/205_4.pkl.gz
<gzip _io.BufferedWriter name='../outTrainVoc/205_4.pkl.gz' 0x7f5d800db940>
160_3.tif
../icdar2013_benchmarking_dataset/160_3.tif
../icdar2013_benchmarking_dataset/160_3.tif
[[0.00438748 0.00786824 0.01020646 ... 0.00335928 0.00381268 0.00260311]]
../outTrainVoc/160_3.pkl.gz
<gzip _io.BufferedWriter name='../outTrainVoc/160_3.pkl.gz' 0x7f5d800dbf60>
170_4.tif
../icdar2013_benchmarking_dataset/170_4.tif
../icdar2013_benchmarking_dataset/170_4.tif
[[0.00581657 0.00820588 0.00954658 ... 0.00780554 0.00333351 0.00263214]]
../outTrainVoc/170_4.pkl.gz
<gzip _io.BufferedWriter name='../outTrainVoc/170_4.pkl.gz' 0x7f5d800db128>
113_1.tif
../icdar2013_benchmarking_dataset/113_1.tif
../icdar2013_benchmarking_dataset/113_1.tif
[[0.00560738 0.00728635 0.01078693 ... 0.0039453  0.00405639 0.00322077]]
../outTrainVoc/113_1.pkl.gz
<gzip _io.BufferedWriter name='../outTrainVoc/113_1.pkl.gz' 0x7f5d800d

[[0.00425231 0.007188   0.01032851 ... 0.00358115 0.00422435 0.00273851]]
../outTrainVoc/050_4.pkl.gz
<gzip _io.BufferedWriter name='../outTrainVoc/050_4.pkl.gz' 0x7f5d800db9b0>
078_4.tif
../icdar2013_benchmarking_dataset/078_4.tif
../icdar2013_benchmarking_dataset/078_4.tif
[[0.00513093 0.0072006  0.0099893  ... 0.00508816 0.00460329 0.00268189]]
../outTrainVoc/078_4.pkl.gz
<gzip _io.BufferedWriter name='../outTrainVoc/078_4.pkl.gz' 0x7f5d800dbac8>
204_3.tif
../icdar2013_benchmarking_dataset/204_3.tif
../icdar2013_benchmarking_dataset/204_3.tif
[[0.00459077 0.00766634 0.01045573 ... 0.0036201  0.004477   0.00272382]]
../outTrainVoc/204_3.pkl.gz
<gzip _io.BufferedWriter name='../outTrainVoc/204_3.pkl.gz' 0x7f5d800dbf60>
026_3.tif
../icdar2013_benchmarking_dataset/026_3.tif
../icdar2013_benchmarking_dataset/026_3.tif
[[0.00423303 0.00762092 0.01089492 ... 0.00351606 0.00396434 0.00279185]]
../outTrainVoc/026_3.pkl.gz
<gzip _io.BufferedWriter name='../outTrainVoc/026_3.pkl.gz' 0x7f5d800d

[[0.00558852 0.00928607 0.0109763  ... 0.00380088 0.00453686 0.00389592]]
../outTrainVoc/071_4.pkl.gz
<gzip _io.BufferedWriter name='../outTrainVoc/071_4.pkl.gz' 0x7f5d800db940>
025_4.tif
../icdar2013_benchmarking_dataset/025_4.tif
../icdar2013_benchmarking_dataset/025_4.tif
[[0.00420291 0.00752011 0.01032867 ... 0.00356301 0.00481752 0.00324231]]
../outTrainVoc/025_4.pkl.gz
<gzip _io.BufferedWriter name='../outTrainVoc/025_4.pkl.gz' 0x7f5d800db518>
151_2.tif
../icdar2013_benchmarking_dataset/151_2.tif
../icdar2013_benchmarking_dataset/151_2.tif
[[0.00478085 0.00812901 0.01041873 ... 0.00350534 0.00350799 0.00276711]]
../outTrainVoc/151_2.pkl.gz
<gzip _io.BufferedWriter name='../outTrainVoc/151_2.pkl.gz' 0x7f5d800db4e0>
101_4.tif
../icdar2013_benchmarking_dataset/101_4.tif
../icdar2013_benchmarking_dataset/101_4.tif
[[0.00572139 0.00752849 0.00975597 ... 0.00359651 0.00392165 0.00303104]]
../outTrainVoc/101_4.pkl.gz
<gzip _io.BufferedWriter name='../outTrainVoc/101_4.pkl.gz' 0x7f5d800d

In [6]:
def load_pickle(fileName):
    'Returns the object stored inside the pickle file'
    if fileName.endswith('pkl.gz'):
        with gzip.open(fileName, 'rb') as f:
            desc = pickle.load(f)
    
    return desc

In [15]:
## Before calling this function
## verify that supervectors for train and test have been computed and are present as pickle files in- 
## -folder ../outTrainVoc and ../outTestVoc
def evaluate():
    'We evaluate using TOP-1 evaluation matrix'
    # X is the supervector belonging to test document
    # Y is the supervector belonging to train document
    testDir = '../outTestVoc/'
    Dir = '../outTestVoc/'
    
    total_test_files = 0
    correctly_classified = 0 # Denoted total no. of correctly classified test files
    
    for testFile in os.listdir(testDir):

        label_test = testFile[0:3] # First three letters denote the writer-ID
        # print ('testfile label: %s' %(label_test))
        # X is the supervector belonging to test document
        cos_dist = []
        if testFile.endswith('pkl.gz'):
            X = load_pickle(os.path.join(testDir, testFile))
            total_test_files += 1
        else:
            continue
        for file in os.listdir(Dir):
            # Y is the supervector belonging to train document
            if file.endswith('pkl.gz'):
                Y = load_pickle(os.path.join(Dir, file))
            else:
                continue
            
            # Computing cosine distance between X and Y supervectors
            dist = spdistance.cosine(X, Y)
            cos_dist.append((dist, file))
        
        # sorting cos_dist on the basis of distance
        list.sort(cos_dist)
        # Extracting the v
        # print ("Top-6 Matches: ")
        # print (cos_dist[0:7])
        
        pred_label = cos_dist[1][1] # looking at the 2nd nearest as 1st one is the same document
        pred_label = pred_label[0:3] # First three letters denote the writer-ID
        
        if (pred_label == label_test):
            correctly_classified += 1
        
    return correctly_classified/total_test_files

In [16]:
Top_1_accuracy = evaluate()
print ('TOP-1 Accuracy: %f' %(Top_1_accuracy))

TOP-1 Accuracy: 0.972500
